In [1]:
from htmd import *

Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049


New HTMD version (1.3.1) is available. You are currently on (1.0.26). Use 'conda update htmd' to update to the new version.


In [2]:
mol = Molecule('3PTB')
mol.filter("protein")

2016-06-30 02:54:08,988 - htmd.molecule.molecule - INFO - Attempting PDB query for 3PTB
2016-06-30 02:54:10,094 - htmd.molecule.molecule - INFO - Removed 72 atoms. 1629 atoms remaining in the molecule.


In [3]:
copied = mol.copy()

In [4]:
copied.filter('resname ARG LYS ASP GLU HIS CYS TYR')

2016-06-30 02:54:10,275 - htmd.molecule.molecule - INFO - Removed 1175 atoms. 454 atoms remaining in the molecule.


In [5]:
rid = np.unique(copied.get('resid')) # ia.resid also works!
rn = copied.get('resname', "name CA")

In [7]:
### This is for the ND1 atom
for resname, resid in zip(rn, rid):
    if resname == "HIS":
        atomlist = [mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name ND1 and resid "+str(resid)), mol.get("name", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name ND1 and resid "+str(resid))]
        print (atomlist[0], "resid = ",resid)
        print(atomlist[1])
        print("test! ", atomlist[0][0], atomlist[1][0])
        for residueID in np.unique(atomlist[0]):
            if residueID != resid:
                print(residueID, np.unique(mol.get("resname", sel="resid "+str(residueID))),)
        
### This is for the NE2 atom
for resname, resid in zip(rn, rid):
    if resname == "HIS":
        atomlist = [mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NE2 and resid "+str(resid)), mol.get("name", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NE2 and resid "+str(resid))]
        print (atomlist[0], "resid = ",resid)
        print(atomlist[1])
        print("test! ", atomlist[0][0], atomlist[1][0])
        for residueID in np.unique(atomlist[0]):
            if residueID != resid:
                print(residueID, np.unique(mol.get("resname", sel="resid "+str(residueID))),)
            

[32 40 40 41 42] resid =  40
['OG' 'ND1' 'NE2' 'N' 'O']
test!  32 OG
32 ['SER']
41 ['PHE']
42 ['CYS']
[ 57  57  57 102 102 214] resid =  57
['N' 'ND1' 'NE2' 'OD1' 'OD2' 'O']
test!  57 N
102 ['ASP']
214 ['SER']
[91 91 91 92 93 94] resid =  91
['O' 'ND1' 'NE2' 'N' 'N' 'N']
test!  91 O
92 ['PRO']
93 ['SER']
94 ['TYR']


[ 40  40  42  42 193] resid =  40
['ND1' 'NE2' 'N' 'O' 'O']
test!  40 ND1
42 ['CYS']
193 ['GLY']
[ 57  57 195 214] resid =  57
['ND1' 'NE2' 'OG' 'O']
test!  57 ND1
195 ['SER']
214 ['SER']
[91 91] resid =  91
['ND1' 'NE2']
test!  91 ND1


In [19]:
#He creado un monstruo... Os lo comento para que os sea mas facil de entender
#La idea es generar una pos_list (que contendra las interacciones entre aa basicos) y una neg_list (que contendra las interacciones entre aa acidos).
#He tomado el esquema que montó Mauro: 
nd1_list = []
ne2_list = []
for resname, resid in zip(rn, rid):

    if resname == "HIS":
        print(resname, resid)
        #primero, seleccionas los residuos que pueden interactuar con el residuo acido/basico de interes mediante puentes de hidrogeno
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG NE NE1 ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name ND1 and resid "+str(resid))
        #eliminamos resultados redundantes
        un = np.unique(a)
        print ("ND1")
        for residueID in un:
            #Uso una arquitectura de try/except para curarme en salud
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                #Como he comentado en el face, hay posiciones de secuencia que contienen dos residuos, en lugar de uno.
                #Para esto, calculamos la distancia, si es mayor que 1 significa que tenemos posicion doble. En ese caso
                #itero sobre las posibles posiciones y si encuentro una que sea un acido (acido en este caso porque tratamos los acidos)
                #me la quedo y la añado al neg_list
                if l > 1:
                    for i in range(0, l):
                        if residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            #Las dos listas generadas siguen un formato de tipo: resname y resid del residuo en el que estoy
                            #y resname y resid del residuo que interactua con el primero
                            if res == "PRO":
                                continue
                            atoms = mol.get("name" , sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG NE NE1 ND1 NH1 OD1 OD2 NZ NH2 and resid "+str(residueID)+" and within 4 of name ND1 and resid "+str(resid))
                            tup = (res, residueID, atoms)
                            nd1_list.append(tup)
                           
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID, atoms)
                else:
                    #Si no hay posiciones con multiples aa, añado residuos a la lista con normalidad
                    if residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        if res == "PRO":
                                continue
                        atoms = mol.get("name" , sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG NE NE1 ND1 NH1 OD1 OD2 NZ NH2 and resid "+str(residueID)+" and within 4 of name ND1 and resid "+str(resid))
                        tup = (res, residueID, atoms)
                        nd1_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID, atoms)
            except:
                #En el caso de que hayan errores, por pantalla aparecera un print indicando en que posicion ocurren
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG NE NE1 ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NE2 and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        print ("NE2")
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    for i in range(0, l):
                        if residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            if res == "PRO":
                                continue
                            atoms = mol.get("name" , sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG NE NE1 ND1 NH1 OD1 OD2 NZ NH2 and resid "+str(residueID)+" and within 4 of name NE2 and resid "+str(resid))
                            tup = (res, residueID, atoms)
                            ne2_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID, atoms)
                else:
                    
                    if residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        if res == "PRO":
                                continue
                        atoms = mol.get("name" , sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG NE NE1 ND1 NH1 OD1 OD2 NZ NH2 and resid "+str(residueID)+" and within 4 of name NE2 and resid "+str(resid))
                        tup = (res, residueID, atoms)
                        ne2_list.append(tup)
                        
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID, atoms)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
        # I'll try to discriminate which nitrogen has to be protonated with something similar as a  "scoring method",
        # based on h-bond capacity (source: http://www.imgt.org/IMGTeducation/Aide-memoire/_UK/aminoacids/charge/)
        # If it can be both donor and acceptor, first the environment is evaluated and then it scores
        # Also, if both nitrogens have an acidic aa near, it protonates that histidine. 
        nd1_negativech = False
        ne2_negativech = False
        nd1_lvl = 0
        ne2_lvl = 0
        # Falten NE, NE1 (Trp), treure SG
        aa_score = {"O": 2, "OD1": 2, "OE1": 2, "OE2": 2, "OD2": 2,"SG": -1, "N": -1, "NE": -1, "NE1": -1, "ND2": -2, "NH1": -2, "NH2": -2, "NZ": -3 }
        # Over-score the negative charges (positive can be identified), and TYM"
        # First, we start by iterating for every aa found around HIS
        for aa in nd1_list:
            # Check if a negative aminoacid is near
            if aa[0] in ["ASP", "GLU", "TYM"]:
                nd1_negativech = True
                nd1_lvl += 2
            # Then, we iterate for every atom found for an specific aminoacid (usually only 1 atom)
            for atom in aa[2]:
                if atom in aa_score:
                    nd1_lvl += aa_score[atom]
                # There are atoms that can be donors or acceptors, so we check if the index is positive or negative to apply a mitigation effect
                elif atom == "OG": #Serine
                    if nd1_lvl <0:
                        nd1_lvl += 2
                    elif nd1_lvl >= 0:
                        nd1_lvl -=1
                elif atom == "OG1": #Threonine
                    if nd1_lvl <0:
                        nd1_lvl += 2
                    elif nd1_lvl >= 0:
                        nd1_lvl -=1
                elif atom == "OH": #Tyrosine
                    if nd1_lvl <0:
                        nd1_lvl += 1
                    elif nd1_lvl >= 0:
                        nd1_lvl -=1
                elif atom == "NE2": # Other HIS or GLN
                    if aa[0] == "GLN":
                        nd1_lvl -= 2
                    elif aa[0] == "HID":
                        nd1_lvl += 1
                    elif aa[0] == "HIE":
                        nd1_lvl -= 1
                    elif aa[0] == "HIP":
                        nd1_lvl -= 1
                    elif aa[0] == "HIS":
                        if nd1_lvl <0:
                            nd1_lvl += 1
                        elif nd1_lvl >= 0:
                            nd1_lvl -=1    
                elif atom == "ND1": # Other HIS or GLN
                    if aa[0] == "HID":
                        nd1_lvl -= 1
                    elif aa[0] == "HIE":
                        nd1_lvl += 1
                    elif aa[0] == "HIP":
                        nd1_lvl -= 1
                    elif aa[0] == "HIS":
                        if nd1_lvl <0:
                            nd1_lvl += 1
                        elif nd1_lvl >= 0:
                            nd1_lvl -=1 
        for aa in ne2_list:
            # Check if a negative aminoacid is near
            if aa[0] in ["ASP", "GLU", "TYM"]:
                ne2_negativech = True
                ne2_lvl += 2
            # Then, we iterate for every atom found for an specific aminoacid (usually only 1 atom)
            for atom in aa[2]:
                if atom in aa_score:
                    ne2_lvl += aa_score[atom]
                # There are atoms that can be donors or acceptors, so we check if the index is positive or negative to apply a mitigation effect
                elif atom == "OG": #Serine
                    if ne2_lvl <0:
                        ne2_lvl += 2
                    elif ne2_lvl >= 0:
                        ne2_lvl -=1
                elif atom == "OG1": #Threonine
                    if ne2_lvl <0:
                        ne2_lvl += 2
                    elif ne2_lvl >= 0:
                        ne2_lvl -=1
                elif atom == "OH": #Tyrosine
                    if ne2_lvl <0:
                        ne2_lvl += 1
                    elif ne2_lvl >= 0:
                        ne2_lvl -=1
                elif atom == "NE2": # Other HIS or GLN
                    if aa[0] == "GLN":
                        ne2_lvl -= 2
                    elif aa[0] == "HID":
                        ne2_lvl += 1
                    elif aa[0] == "HIE":
                        ne2_lvl -= 1
                    elif aa[0] == "HIP":
                        ne2_lvl -= 1
                    elif aa[0] == "HIS":
                        if nd1_lvl <0:
                            ne2_lvl += 1
                        elif nd1_lvl >= 0:
                            ne2_lvl -=1    
                elif atom == "ND1": # Other HIS or GLN
                    if aa[0] == "HID":
                        ne2_lvl -= 1
                    elif aa[0] == "HIE":
                        ne2_lvl += 1
                    elif aa[0] == "HIP":
                        ne2_lvl -= 1
                    elif aa[0] == "HIS":
                        if ne2_lvl <0:
                            ne2_lvl += 1
                        elif ne2_lvl >= 0:
                            ne2_lvl -=1
        # Okay, scoring made, let's decide the protonation!
        if nd1_negativech and ne2_negativech:
            mol.set("resname", "HIP", "resname "+str(resname)+" and resid "+str(resid))   
        if nd1_lvl >= ne2_lvl:
            mol.set("resname", "HID", "resname "+str(resname)+" and resid "+str(resid))
        elif nd1_lvl < ne2_lvl:
            mol.set("resname", "HIE", "resname "+str(resname)+" and resid "+str(resid))
                    
                
        
        
                




HIS 40
ND1
['SER'] 32 ['OG']
['PHE'] 41 ['N']
['CYS'] 42 ['O']
NE2
['CYS'] 42 ['N' 'O']
['GLY'] 193 ['O']
HIS 57
ND1
['ASP'] 102 ['OD1' 'OD2']
['SER'] 214 ['O']
NE2
['SER'] 195 ['OG']
['SER'] 214 ['O']
HIS 91
ND1
['SER'] 93 ['N']
['TYR'] 94 ['N']
NE2


In [18]:
print(mol.get("resname", sel="resid 40"))
print(mol.get("resname", sel="resid 57"))
print(mol.get("resname", sel="resid 91"))
#Worked!

['HIE' 'HIE' 'HIE' 'HIE' 'HIE' 'HIE' 'HIE' 'HIE' 'HIE' 'HIE']
['HID' 'HID' 'HID' 'HID' 'HID' 'HID' 'HID' 'HID' 'HID' 'HID']
['HID' 'HID' 'HID' 'HID' 'HID' 'HID' 'HID' 'HID' 'HID' 'HID']
